# MedNow Clinics — Excel Ingest, Filter, Map, and Export (pandas)

This notebook reproduces the Polars pipeline using **pandas**.

Steps:
1. Read multi-sheet Excel (skip **Monthly Summary**).
2. Normalize headers.
3. Drop PHI name fields.
4. Filter Humana insurance.
5. Split **procedure** into **CPT_Code** and **Procedure_Description**, add **Code_Core**.
6. Merge with `service_map` on CPT_Code, fallback to Code_Core, else Unknown.
7. Write CSV and Parquet.


## Imports

In [1]:
import pandas as pd
from pathlib import Path
import openpyxl

## Config

In [2]:
INPUT_XLSX = Path(r"C:\Users\pirat\Dropbox\Consulting Inc\MedNow\MNC Athena Activity - Monthly 2024 & YTD June 2024 - As of 10.2.25 - Copy.xlsx")
SERVICE_MAP_XLSX = Path(r"C:\Users\pirat\Dropbox\Consulting Inc\Josh Perkins Data\CPT_Code_Service_Map.xlsx")
SERVICE_MAP_SHEET = "Sheet1"
OUT_CSV = Path(r"C:\Users\pirat\Dropbox\Consulting Inc\MedNow\Humana Analysis\mednow_athena_2024_2025_ytd_06_humana.csv")  
# OUT_PARQUET = Path(r"C:\path\to\mednow_humana.parquet")
SKIP_SHEETS = {"Monthly Summary"}
NAME_COLS = ["last_name","first_name","middle_initial","name_suffix"]
RENAME_MAP: dict[str, str] = {
    "service dept": "Service_Dept",
    "rendering provider": "Rendering_Provider",
    "procedure": "Procedure",
    "insurancepackage": "Insurance_Package",
    "last name": "last_name", # Drop
    "first name": "first_name", # Drop
    "middle initial": "middle_initial", # Drop
    "name suffix": "name_suffix", # Drop
    "patient id": "Patient_ID",
    "patient date of birth": "Patient_DOB", # Drop
    "service date": "Service_Date",
    "#chg": "Num_Chg",          # counts
    "chg": "Chg",               # raw charge
    "netpmt": "Net_Pmt",         # net payment
    "contract": "Contract",
    "free": "Free",
    "baddebt": "Bad_Debt",
    "pre-collect": "Pre_Collect",
    "collect": "Collect",
    "recovery": "Recovery",
    "fee": "Fee",
    "cap": "Cap",
    "globl": "Globl",
    "w/hold": "W_Hold",
    "interest": "Interest",
    "adjoth": "Adj_Oth",
    "incentive": "Incentive",
    "vendor_adjustments": "Vendor_Adjustments",
    "net xfer": "Net_Transfer",
    "netreceivable": "Net_Receivable",
}

## Helpers

In [3]:
def normalize_colname(name: str) -> str:
    out = name.strip().lower()
    for ch in ["/"," ","-","#","(",")","[","]"]:
        out = out.replace(ch, "_")
    while "__" in out:
        out = out.replace("__", "_")
    return out.strip("_")


def apply_renames(df: pd.DataFrame) -> pd.DataFrame:
    lower_map = {k.lower(): v for k, v in RENAME_MAP.items()}
    mapping = {}
    for c in df.columns:
        std = normalize_colname(c)
        std = lower_map.get(c.lower(), lower_map.get(std, std))
        mapping[c] = std
    return df.rename(columns=mapping)


def list_sheet_names(xlsx_path: Path) -> list[str]:
    wb = openpyxl.load_workbook(xlsx_path, read_only=True, data_only=True)
    return [s for s in wb.sheetnames if s not in SKIP_SHEETS]


def read_all_sheets_df(xlsx_path: Path) -> pd.DataFrame:
    dfs = []
    for s in list_sheet_names(xlsx_path):
        df = pd.read_excel(xlsx_path, sheet_name=s)
        if df.empty:
            continue
        df = apply_renames(df)
        df["source_sheet"] = s
        df["source_file"] = xlsx_path.name
        dfs.append(df)
    if not dfs:
        return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True)

## Ingest MedNow Excel

In [4]:
df = read_all_sheets_df(INPUT_XLSX)

## Drop PHI and filter to Humana

In [5]:
df_filtered = df.copy()
df_filtered = df_filtered.drop(columns=[c for c in NAME_COLS if c in df.columns])
df_filtered = df_filtered[df_filtered["Insurance_Package"].str.contains("humana", case=False, na=False)].copy()

## Split Procedure into CPT_Code and Procedure_Description, derive Code_Core

In [6]:
proc_split = df_filtered["Procedure"].str.split(": ", n=1, expand=True)
df_filtered["CPT_Code"] = proc_split[0].str.replace(",","-", regex=False)
df_filtered["Procedure_Description"] = proc_split[1]
df_filtered["Code_Core"] = df_filtered["CPT_Code"].str[:5]

cols = list(df_filtered.columns)
pidx = cols.index("Procedure")
new_order = cols[:pidx+1] + ["CPT_Code","Code_Core","Procedure_Description"] + [c for c in cols[pidx+1:] if c not in ["CPT_Code","Code_Core","Procedure_Description"]]
df_filtered = df_filtered[new_order]

## Load service_map

In [7]:
service_map = pd.read_excel(SERVICE_MAP_XLSX, sheet_name=SERVICE_MAP_SHEET)
service_map.head(), service_map.columns

(  Code Full Code Core Modifier  CPT/HCPCS Description         UHC Category  \
 0     0001U     0001U       00  CPT/HCPCS Description  CLINICAL LABORATORY   
 1     0002M     0002M       00       RBC DNA HEA 35 A  CLINICAL LABORATORY   
 2     0002U     0002U       00       LIVER DIS 10 ASS  CLINICAL LABORATORY   
 3     0003M     0003M       00       ONC CLRCT QUAN 3  CLINICAL LABORATORY   
 4     0003U     0003U       00       LIVER DIS 10 ASS  CLINICAL LABORATORY   
 
   Cigna Category    Category Name Sub-Category Name  
 0            NaN  Pathology & Lab    Lab - Clinical  
 1            NaN  Pathology & Lab    Lab - Clinical  
 2            NaN  Pathology & Lab    Lab - Clinical  
 3            NaN  Pathology & Lab    Lab - Clinical  
 4            NaN  Pathology & Lab    Lab - Clinical  ,
 Index(['Code Full', 'Code Core', 'Modifier', 'CPT/HCPCS Description',
        'UHC Category', 'Cigna Category', 'Category Name', 'Sub-Category Name'],
       dtype='object'))

## Join service_map by CPT_Code with fallback to Code_Core

In [8]:
# payload columns to bring over
PAYLOAD = [c for c in service_map.columns if c not in {"Code Full","Code Core"}]

# 1) exact CPT map (unique by assumption)
full_map = service_map.set_index("Code Full")[PAYLOAD]

# 2) core map using first occurrence only
core_map = service_map.drop_duplicates(subset=["Code Core"], keep="first").set_index("Code Core")[PAYLOAD]

joined = df_filtered.copy()

# join exact CPT first
joined = joined.join(full_map, on="CPT_Code")

# join core as fallback (suffix to coalesce)
joined = joined.join(core_map, on="Code_Core", rsuffix="_core")

# coalesce exact->core->Unknown
for c in PAYLOAD:
    joined[c] = joined[c].combine_first(joined.get(f"{c}_core")).fillna("Unknown")

# clean up
joined = joined.drop(columns=[f"{c}_core" for c in PAYLOAD if f"{c}_core" in joined])


In [9]:
# Add Total_Paid and Visit_ID with renamed columns

# columns to sum for Total_Paid
cols_to_sum = [
    "Adj_Oth", "Bad_Debt", "Cap", "Collect", "Contract", "Fee", "Free",
    "Globl", "Incentive", "Interest", "Net_Pmt", "Net_Transfer",
    "Pre_Collect", "Recovery", "Vendor_Adjustments", "W_Hold"
]

# ensure numeric types (convert errors to NaN, fill with 0)
joined[cols_to_sum] = joined[cols_to_sum].apply(pd.to_numeric, errors="coerce").fillna(0)

# row-wise sum
joined["Total_Paid"] = joined[cols_to_sum].sum(axis=1)

# Visit_ID = Patient_ID-Service_Date
joined["Visit_ID"] = (
    joined["Patient_ID"].astype(str) + "-" + joined["Service_Date"].astype(str)
)

joined[["Patient_ID","Service_Date","Total_Paid","Visit_ID"]].head()


,Patient_ID,Service_Date,Total_Paid,Visit_ID
148,37495,2024-01-10,0.0,37495-2024-01-10
244,37495,2024-01-10,0.0,37495-2024-01-10
245,104759,2024-01-03,0.0,104759-2024-01-03
246,91424,2024-01-05,0.0,91424-2024-01-05
396,37495,2024-01-10,0.0,37495-2024-01-10


## Write outputs

In [10]:
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
joined.to_csv(OUT_CSV, index=False)
# joined.to_parquet(OUT_PARQUET, index=False)
# OUT_CSV, OUT_PARQUET

In [12]:
df.to_csv(r"C:\Users\pirat\Dropbox\Consulting Inc\MedNow\mednow_athena_2024_2025_ytd_06_full.csv", index=False)

## QA checks